In [ ]:
import numpy as np
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torchvision import datasets, transforms

from sklearn.metrics import accuracy_score

In [ ]:
train_data = datasets.CIFAR10(root="./data", train=True, download=True, transform=transforms.ToTensor())
# download - скачать с сервера на диск
test_data = datasets.CIFAR10(root="./data", train=False, download=True, transform=transforms.ToTensor())

DataLoader - механизм, который на каждой итерации обучения нейросети будет выделять из датасета батч картинок и подавать сети на вход.
Важно - перемешивать данные перед train.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=len(test_data), shuffle=False)

In [ ]:
dataiter = iter(train_loader)
# батч картинок и батч ответов к картинкам
images, labels = dataiter.next()

In [ ]:
images.shape, labels.shape

In [ ]:
def show_imgs(imgs, labels):
    f, axes = plt.subplots(1, 10, figsize=(30, 5))
    for idx, axis in enumerate(axes):
        axes[idx].imshow(np.squeeze(np.transpose(imgs[idx].numpy(), (1, 2, 0))), cmap='gray')
        axes[idx].set_title(labels[idx].numpy())
    plt.show()
    
show_imgs(images, labels)

Таблица соответствий номеров ответов и классов:

| Label        | Class           |
| ------------- |:-------------:|
| 0     | Самолет |
| 1    | Автомобиль      | 
| 2 | Птица      |
| 3 | Кошка      |
| 4 | Олень      |
| 5 | Собака      |
| 6 | Лягушка     |
| 7 | Лошадь     |
| 8 | Корабль     |
| 9 | Грузовик     |


Импортируем нужные модули для обучения сети:

In [ ]:
# модуль, где определены слои для нейронных сетей
import torch.nn as nn
# модуль, где определены активации для слоев нейронных сетей
import torch.nn.functional as F

### Обучение и тест базовой сети

Для начала построим двухслойную полносвязную нейронную сеть, обучим ее и посчитаем метрику accuracy на тестовой выборке:

In [ ]:
# класс для удобного перевода картинки из двумерного объекта в вектор
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = Flatten()
        self.fc1 = nn.Linear(32*32*3, 256) # полносвязный слой в pytorch
        self.fc2 = nn.Linear(256, 10) # классификация на 10 классов
        
    def forward(self, x):
        # forward pass сети
        
        # переводим входной объект из картинки в вектор
        x= self.flatten(x)
        # умножение на матрицу весов 1 слоя и применение функции активации
        x = F.relu(self.fc1(x))
        # умножение на матрицу весов 2 слоя и применение функции активации
        x = F.softmax(self.fc2(x))
        
        return x

Вынесем код обучения сети в отдельную функцию, чтобы каждый раз при изменении сети не копировать его. Код обучения мы менять не будем

In [ ]:
def train(net, n_epoch=2):
    # выбираем функцию потерь
    loss_fn = torch.nn.CrossEntropyLoss()
    
    # выбираем алгоритм оптимизации и learning_rate
    learning_rate = 1e-3
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    
    # обучаем сеть 2 эпохи
    for epoch in tqdm_notebook(range(n_epoch)):
        running_loss = 0.0
        train_dataiter = iter(train_loader)
        for idx, batch in enumerate(tqdm_notebook(train_dataiter)):
            # так получаем текущий батч
            X_batch, y_batch = batch
            
            # обнуляем веса
            optimizer.zero_grad()
            
            # forward pass (получение ответов на батч картинок)
            y_pred = net(X_batch)
            # вычисление потерь от выданных сетью ответов и правильных ответов на батч
            loss = loss_fn(y_pred, y_batch)
            # backpropagation (вычисление градиентов)
            loss.backward()
            # обновление весов сети
            optimizer.step()
            
            # выведем текущий loss
            running_loss += loss.item()
            # выведем качество каждые 500 батчей
            if idx % 500 == 499:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, idx + 1, running_loss / 500))
                running_loss = 0.0
                
    print('Обучение закончено')
    return net

In [ ]:
# объявляем сеть
net = SimpleNet()
net = train(net)

Посчитаем accuracy на test:

In [ ]:
test_dataiter = iter(test_loader)
images, labels = test_dataiter.next()

In [ ]:
accuracy_score(labels.numpy(), np.argmax(net.forward(images).detach().numpy(), axis=1))

### Обучение сверточной сети

Полносвязна нейронная сеть не очнь хорошо справляется с поставленной задачей. Попробуем объявить и обучить свеврточную нейронную сеть.

Наша нейронная сеть будет содержать три слоя: два сверточных и один полносвязный. В качестве функции активации для внутренних слоев будем использовать relu, для последнего слоя - softmax.

In [ ]:
# класс для удобного перевода картинки из двумерного объекта в вектор
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=3)
        self.conv2 = nn.Conv2d(20, 3, kernel_size=3)
        self.flatten = Flatten()
        self.fc = nn.Linear(2352, 10)
        
    def forward(self, x):
        # forward pass сети
        # умножение на матрицу весов 1 слоя и применение функции активации
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.flatten(x)
        # умножение на матрицу весов 2 слоя и применение функии активации
        x = F.softmax(self.fc(x))
        
        return x

In [ ]:
net = ConvNet()

In [ ]:
net = train(net)

In [ ]:
test_dataiter = iter(test_loader)
images, labels = test_dataiter.next()

In [ ]:
accuracy_score(labels.numpy(), np.argmax(net.forward(images).detach().numpy(), axis=1))

### BachNormalization

Одно из улучшений нейронной сети -- Батч Норм (BatchNorm).

Смысл Батч Норма состоит в следующем: как вы знаете, когда мы обучаем нейронную сеть, мы обучаем ее батчами. Каждый батч картинок имеет разное распределение. Когда батч проходит через первый слой сети, выход первого слоя сети тоже имеет какое-то распределение. И для всех батчей оно немного разное.

Таким образом, второй слой сети каждый раз принимает на вход данные, распределенные по-разному. И при обработке первого батча картинок веса сети подстраиваются под распределение картинок этого батча, а при обработке второго должны подстраиваться под другое распределение второго батча. Такое "метание" плохо сказывается на обучении сети, оно становится менее стабильным и в итоге сеть обучается хуже.

Батч норм -- это специальный слой сети, который используется после каждого слоя нейронной сети (кроме последнего). Он нормализует выходы слоев. Таким образом, следуюшему слою нейронной сети каждый раз передаются данные, распределенные примерно одинаково.

Чтобы лучше осознать полезность батч норма, можно вспомнить, что при изучении КНН мы говорили о том, что перед обучением модели признаки можно нормировать. Если есть два признака (например, возраст и доход), один из которых представлен десятками, а другой -- десятками тысяч, то для лучшего обучения модели их лучше привести к одному виду. Так и тут -- для лучшего обучения нейронной сети батчи лучше привести к одному виду.

Более подробно прочитать про батч норм можно тут: http://neerc.ifmo.ru/wiki/index.php?title=Batch-normalization

Напишем класс нейронной сети из трех слоев с батч нормализацией. За основу возьмем сеть, написанную выше: три слоя, 256, 64 и 10 нейронов в каждом.

Задание: реализуйте код нейронной сети с батч нормами. Допишите код в ячейке ниже в недостающих местах и запустите обучение сети.

In [ ]:
# класс для удобного перевода картинки из двумерного объекта в вектор
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = Flatten()
        self.fc1 = <первый линейный слой размерности <размер вектора картинки>*256>
        # задаем слой батч норма для выхода слоя сети с 256 нейронами
        self.bn1 = nn.BatchNorm1d(256) 
        self.fc2 = <первый линейный слой размерности 256*64>
        self.bn2 = <второй батч норм для выхода слоя сети с 64 нейронами>
        self.fc3 = <первый линейный слой размерности 64*10>
    
    def forward(self, x):
        # forward pass сети

        # переводим входной объект из картинки в вектор
        x = self.flatten(x)
        # применение слоев и батч нормов по очереди:
        <пропустите х через все слои и активации: fc1-relu-bn1-fc2-relu-bn2-fc3-softmax>

        return x

In [ ]:
# объявляем сеть
net = SimpleNN()

In [ ]:
# не изменяйте эту ячейку!
# эта ячейка проверяет ваш код на правильность
# если при запуске ячейка выдает ошибку, то у вас в коде ошибка

assert len(net.state_dict().keys()) == 16, "Неверное количество слоев. \
Проверьте, что в вашей сети три слоя и два батч-норма после первого и второго слоя."

fc1 = list(net.state_dict().keys())[0]
fc2 = list(net.state_dict().keys())[7]
fc3 = list(net.state_dict().keys())[14]

assert net.state_dict()[fc1].shape == (256, 3072), "Неверные размерности первого слоя. \
Должно быть размер вектора картинки * 256"
assert net.state_dict()[fc2].shape == (64, 256), "Неверные размерности второго слоя. \
Должно быть 256 * 64"
assert net.state_dict()[fc3].shape == (10, 64), "Неверные размерности третьего слоя. \
Должно быть 64 * 10"

In [ ]:
net = train(net)

In [ ]:
test_dataiter = iter(test_loader)
images, labels = test_dataiter.next()

In [ ]:
# убедиться, что accuracy увеличился! Batch Norm действительно работает.
accuracy_score(labels.numpy(), np.argmax(net.forward(images).detach().numpy(), axis=1))

### Подбор гиперпараметров

Наверное, вы уже заметили, что у нас в модели довольно много гиперпараметров. Например: количество слоев, размеры каждого слоя, learning rate в градиентном спуске. Как и в не нейронных моделях машинного обучения, эти гиперпараметры можно подбирать. Этим мы сейчас и займемся.

Перенесем сюда код класса нейронной сети и функции обучения.

Задание: проведите несколько экспериментов (минимум 3) обучения сети с разными параметрами. Варьируйте количество слоев, количество нейронов в слоях (у нас выше были 256 и 64, попробуйте других числа), различные функции активации после внутренних слоев (после последнего оставьте softmax) и разный learning rate. Опишите свои выводы на основе проделанных экспериментов в отдельной ячейке ниже.

P.S. разные функции активации, как и relu, можно найти в torch.functional, который у нас импортирован как F.

In [ ]:
# примеры функций активации
F.relu
F.celu
F.elu
F.leaky_relu
F.prelu

Описать выводы на основе проделанных экспериментов тут. Как повляили различные значения гиперпараметров на итоговую величиную accuracy? Как вы думаете, почему?

In [ ]:
# класс для удобного перевода картинки из двумерного объекта в вектор
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = Flatten()
        self.fc1 = <первый линейный слой какой-нибудь размерности>
        <еще слои, батч нормы на ваш выбор>
        
    
    def forward(self, x):
        # forward pass сети

        # переводим входной объект из картинки в вектор
        x = self.flatten(x)
        <примените ваши слои, объявленные в init. Не забудьте про функици активации!>
        return x

In [ ]:
def train(net):
  # выбираем функцию потерь
  loss_fn = torch.nn.CrossEntropyLoss()

  # выбираем алгоритм оптимизации и learning_rate
  learning_rate = <попробуйте обучить сеть с разными значениями learning rate. Например, 0.1, 0.01, ...>
  optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

  # обучаем сеть 2 эпохи
  for epoch in tqdm_notebook(range(2)):

      running_loss = 0.0
      train_dataiter = iter(train_loader)
      for i, batch in enumerate(tqdm_notebook(train_dataiter)):
          # так получаем текущий батч
          X_batch, y_batch = batch
          
          # обнуляем веса
          optimizer.zero_grad()

          # forward pass (получение ответов на батч картинок)
          y_pred = net(X_batch)
          # вычисление лосса от выданных сетью ответов и правильных ответов на батч
          loss = loss_fn(y_pred, y_batch)
          # bsckpropagation (вычисление градиентов)
          loss.backward()
          # обновление весов сети
          optimizer.step()

          # выведем текущий loss
          running_loss += loss.item()
          # выведем качество каждые 500 батчей
          if i % 500 == 499:
              print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 500))
              running_loss = 0.0

  print('Обучение закончено')
  return net

In [ ]:
net = SimpleNN()
net = train(net)

In [ ]:
test_dataiter = iter(test_loader)
images, labels = test_dataiter.next()

In [ ]:
accuracy_score(labels.numpy(), np.argmax(net.forward(images).detach().numpy(), axis=1))

Это далеко не все способы улучшить сеть. Есть еще много. Например, Dropout (дропаут) помогает бороться с переобчением сетей. Почитать о нем можно тут: Дропаут на викиконспектах.

Вообще, подбор параметров для обучения нейронной сети -- очень сложная задача. И чем задача сложнее (машинный перевод, генерация изображений и т.д.), тем нейронные сети становятся глубже и имеют все больше параметров. Для того, чтобы хорошо обучить сеть для сложной задачи, нужно подобрать архитектуру, количество слоев, количество нейронов в слое, правильный метод обучения -- learning rate, количество эпох и еще много других параметров. Это сделать весьма непросто.